# Mocking and Patching Objects for Testing

### Topics

- the motivation behind mocking/facking/patching objects
- What is a Magic Mock?
- use Mock to imitate objects in your tests
- check usage data to understand how you use your objects
- customize your mock objects' return values and side effects
- patch() objects throughout your codebase
- common mock problems and avoiding them

## Motivation

- isolated problems are easier to diagnose and solve
- if a test fails, diagnosing/debugging many interrelated components can be very difficult
    - e.g., why the engine of a gasoline car not firing
- we often want to isolate items and testing environments by providing simplified imitations (facking real objects)
- two reasons to replace actual code/API with imitation or **mock/fake** objects
    1. isolate a unit under test - create collaborating classes and functions so we can test one unknown component
    2. test code that requires an object that is either expensive or risky to use;
       - things like shared databases, filesystems, and cloud infrastructures can be very expensive to setup and tear down for testing

### Problems

- two immediate problems we've been facing when solving Kattis problems are:
1. how to programmatically assert what result the function printed to standard output?
2. how to automate data from standard input with out manually entering the input?

In [ ]:
# Quick demo of patching stdout object to write to a file
# instead of the console.
from sys import stdout

In [ ]:
# by default, stdout is a file object that writes to the console
stdout.write('Hello World!\n')

In [ ]:
save_stdout = stdout # save the original stdout object
stdout = open('log.txt', 'w')
stdout.write('This is a log file\n')
stdout.close()

In [ ]:
! cat log.txt

In [ ]:
stdout = save_stdout # restore the original stdout object
stdout.write('Back to the console\n')

In [ ]:
def print():
    x = input('enter')

In [ ]:
x = print()

## Imitating objects using Mocks

- based on: [https://realpython.com/python-mock-library/](https://realpython.com/python-mock-library/) and [https://docs.python.org/3/library/unittest.mock.html](https://docs.python.org/3/library/unittest.mock.html)
- `unittest.mock` module provides Mock base class for mocking objects
- you can pass mock objects as arguments to functions
- assign/patch other objects
- when substituting an object in your code, the Mock must look like the real object it is replacing
    - mock objects must have the same members (attributes and methods) that are being tested
    - e.g., if you're mocking `json` library and your program calls `dumps()`, then the mock object must also contain `dumps()` 
- Mock must simulate any object that it replaces
    - Mock creates attributes/members when you access them dynamically!
- Mock methods can take whatever arguments you provide but always return Mock object

In [ ]:
from unittest.mock import Mock

In [ ]:
help(Mock)

In [ ]:
mock = Mock()

In [ ]:
mock

In [ ]:
mock.some_attribute

In [ ]:
mock.do_something()
# mock methods return Mock object

In [ ]:
# let's use this Hello class to use some mocking/patching concepts
class Hello(object):
    def __init__(self, msg="Hello there") -> None:
        self.__msg = msg
        
    def greet(self) -> str:
        return self.__msg
    
    def other_method(self) -> str:
        # 
        return "Other method called"

In [ ]:
hi = Hello()

In [ ]:
print(hi.greet())

In [ ]:
mock_hi = Mock()

In [ ]:
# returns a Mock object by default
mock_hi.greet()

In [ ]:
mock_hi.greet.return_value = 'Hello there'

In [ ]:
print(mock_hi.greet())

In [ ]:
type(mock_hi)

In [ ]:
type(mock_hi.greet)

In [ ]:
assert mock_hi.greet() == 'Hello there'

In [ ]:
mock_hi.greet.assert_called()

In [ ]:
mock_hi.greet.call_count

In [ ]:
mock_hi.__msg = 'Hi there'

In [ ]:
# greet() doesn't return __msg attribute
mock_hi.greet()

In [ ]:
mock_hi.__msg

In [ ]:
# let's see the Python std JSON library
import json

In [ ]:
help(json)

In [ ]:
# dump requires two positional arguments - see '*' in dump definition
data = json.dump()

In [ ]:
# Patching JSON object
json = Mock()

In [ ]:
json.dump()
# takes any or no arguments; returns Mock object

### Assertions and Inspection

- Mock instances store data on how you used them
    - e.g., if you called a method, the instance stores information on how you called the method, how many times, and so on...
- examples of how to use this information

In [ ]:
from unittest.mock import Mock

In [ ]:
# create a mock object
json = Mock()

In [ ]:
json.loads('{"key": "value"}')

In [ ]:
# we know that we called loads() 
# so we can make assertions to test that expectation
json.loads.assert_called()

In [ ]:
json.loads.assert_called_once()

In [ ]:
json.loads.assert_called_with('{"key": "value"}')

In [ ]:
# this will raise an AssertionError
json.loads.assert_called_with('{"key1": "value1"}')

In [ ]:
json.loads.assert_called_once_with('{"key": "value"}')

In [ ]:
json.loads('{"key": "value"}')

In [ ]:
json.loads.assert_called_once()

In [ ]:
json.loads.assert_not_called()

### Mock special attributes

- Mock objects have several special attributes, e.g.:
- `call_count`
- `call_args`
- `method_calls`
- helps you understand how your application used an object

In [ ]:
from unittest.mock import Mock

In [ ]:
json = Mock()

In [ ]:
json.loads('{"key": "value"}')

In [ ]:
json.loads('{"key1": "value1"}')

In [ ]:
# number of times you called loads
json.loads.call_count

In [ ]:
# the last loads call
json.loads.call_args

In [ ]:
# list of calls to json's methods (recursively)
json.method_calls

### Managing a Mock's return value

- mocks let you control your code's behavior during tests
- one important aspect of the testing is to control object's behavior (methods) and their return values
- e.g., write a function that determines whether today is a weekday
    - depending on what day the test is run, you may get different results from your `calendar` utility
- you can mock `datetime` and set the `.return_value` for `.today()` to a day that you choose

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
def is_weekday():
    today = datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return 0 <= today.weekday() < 5


In [ ]:
# Test if today is a weekday
assert is_weekday()
# if you run the test on weekend, you'll get an AssertionError

In [ ]:
# pick couple of days from the past!
tuesday = datetime(year=2023, month=1, day=3)
saturday = datetime(year=2023, month=1, day=7)

In [ ]:
# Mock datetime to control today's date
datetime = Mock()

In [ ]:
def is_weekday():
    today = datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return 0 <= today.weekday() < 5

In [ ]:
is_weekday()

In [ ]:
# Mock .today() to return Tuesday
datetime.today.return_value = tuesday

In [ ]:
datetime.today.return_value

In [ ]:
datetime.today().day

In [ ]:
datetime.today().month

In [ ]:
datetime.today().weekday() # Tuesday is 1

In [ ]:
# Test tuesday is a weekday
assert is_weekday()

In [ ]:
# Mock .today() to return Saturday
datetime.today.return_value = saturday

In [ ]:
# Test Saturday is not a weekday
assert is_weekday()

In [ ]:
# but saturday is not weekday!
assert not is_weekday()

### Managing Mock's side effects

- you can control code's behavior by specifying a mocked function's side effects (https://docs.python.org/3/library/unittest.mock.html#unittest.mock.Mock.side_effect)
- side effect defines what happens when you call the mocked function
- you can use side effects to mock the return values, e.g.
- see `src/mocking/mock_demo.py` file for a full demo

In [ ]:
mock = Mock()

In [ ]:
# using side_effect to raise an exception
mock.side_effect = Exception('Boom')

In [ ]:
mock()

In [ ]:
# using side_effect to return a sequence of values
mock = Mock()
mock.side_effect = [3, 2, 1]

In [ ]:
# call mock three times to return each side effect
mock()

### Configuring Your Mock

- can configure Mock to set some of the object's behaviors and attributes
- two ways to configure Mock:
    - when you create it (during init)
    - when you use `.configure_mock()` method on Mock object

In [ ]:
mock = Mock(side_effect=Exception)

In [ ]:
# the return value is an exception
mock()

In [ ]:
mock = Mock(name='Mocking with Python')

In [ ]:
mock.name

In [ ]:
mock = Mock(return_value=True)

In [ ]:
mock()

In [ ]:
# using .configure_mock()
mock = Mock()

In [ ]:
mock.configure_mock(return_value=True)

In [ ]:
mock()

In [ ]:
# same as
mock = Mock(return_value='fish')

In [ ]:
mock()

## Patching

- patching makes it easier to Mock objects that are imported from a different module
- `unittest.mock` provides a powerful mechanism for mocking objects called `patch()`
- `patch()` looks up an object in a given module and replaces that object with a Mock
- usually, you use patch() as a decorator or a context manager to provide a scope in which you'll mock the target object
- patching a class replaces the class with `MagicMock` instance
- see these demos: `src/mocking/patch_demo.py` and `src/mocking/patch_demo1.py`

### patch( ) as a Decorator
- use patch() as a decorator to mock an object for the duration of your entire test function
- you can stack multiple patch decorators - see examples here: https://docs.python.org/3/library/unittest.mock.html

In [ ]:
from unittest.mock import patch

In [ ]:
help(patch)

In [ ]:
# Hello class before patched
Hello().greet()

In [ ]:
@patch('__main__.Hello', spec=True)
def test_Hello(Hello_Mock):
    assert Hello_Mock is Hello
    # If the class is instantiated in the code under test 
    # then it will be the return_value of the mock that will be used
    h = Hello_Mock.return_value
    h.greet.return_value = "Howdy!"
    #h.say_hi.return_value = 'hi'
    assert h.greet() == 'Howdy!'
    o = Hello()
    assert o.greet() == 'Howdy!'
    assert Hello().greet() == 'Howdy!'
    assert Hello_Mock.called
    assert Hello.called

In [ ]:
# Hello class patched within the test_Hello function
test_Hello()

In [ ]:
# Hello class after the patch shouldn't change!
Hello().greet()

### patch( ) as a Context Manager

- you only want to mock an object for a part of the test scope
- you're already using too many decorators or parameters which hurts your test readability

In [ ]:
def test_Hello_context():
    with patch('__main__.Hello', spec=True) as Hello_Mock:
        assert Hello_Mock is Hello
        h = Hello_Mock.return_value
        h.greet.return_value = "Good Bye!"
        assert h.greet() == 'Good Bye!'
        assert Hello().greet() == 'Good Bye!'
        assert Hello_Mock.called
        assert Hello.called

In [ ]:
test_Hello_context()

### patching only object's attributes/methods/API

- you can also mock only one method of an object instead of the entire object
- use `patch.object(class, 'class_method')`

In [ ]:
from unittest.mock import patch

In [ ]:
# just mock the greet method
from typing import Callable as callable
@patch.object(Hello, 'greet', spec=True)
def test_greet(mock_greet: callable) -> None:
    Hello.greet.return_value = 'Howdy!'
    # mock_greet is same as Hello.greet
    assert mock_greet() == 'Howdy!'


In [ ]:
test_greet()

In [ ]:
# let's install requests library
! pip install requests

In [ ]:
import requests
from requests.exceptions import Timeout

In [ ]:
help(requests)

In [ ]:
response = requests.get('https://example.com')

In [ ]:
print(response)

In [ ]:
response.status_code

In [ ]:
help(response)

In [ ]:
response.text

In [ ]:
# test 404
@patch.object(requests, 'get') # (module, 'function')
def test_requests_get_404(mock_method):
    url = 'http://localhost/calendar/api/holidays'
    requests.get.return_value = Mock(status_code=404)
    response = requests.get(url)
    mock_method.assert_called_with(url)
    assert(response.status_code == 404)

In [ ]:
test_requests_get_404()

In [ ]:
# test holidays
@patch.object(requests, 'get') # (module, 'function')
def test_requests_get_christmas(mock_get):
    url = 'http://localhost/calendar/api/holidays'
    holiday_json = '{"Christmas: "December 25, 2023"}'
    requests.get.return_value = Mock(status_code=202, return_value=holiday_json)
    response = requests.get(url)
    mock_get.assert_called_with(url)
    assert(response.status_code == 202)
    assert(response.return_value == holiday_json)

In [ ]:
test_requests_get_christmas()

### Patching Standard IO

- if the Class API/function uses print/stdout, input/stdin, it'll be difficult to unittest without patching stdio
- you can patch stdio with StringIO

In [109]:
# StringIO has API similar to stdio
from io import StringIO

In [110]:
help(StringIO)

Help on class StringIO in module io:

class StringIO(_TextIOBase)
 |  StringIO(initial_value='', newline='\n')
 |  
 |  Text I/O implementation using an in-memory buffer.
 |  
 |  The initial_value argument sets the value of object.  The newline
 |  argument is like the one of TextIOWrapper's constructor.
 |  
 |  Method resolution order:
 |      StringIO
 |      _TextIOBase
 |      _IOBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getstate__(...)
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __setstate__(...)
 |  
 |  close(self, /)
 |      Close the IO object.
 |      
 |      Attempting any further operation after the object is closed
 |      will raise a ValueError.
 |      
 |      This method has no effect if the file is already closed.
 |  
 |  getvalue(self, /)
 |      Retrieve the entire contents of the object.
 |  
 |  

In [111]:
import sys

In [112]:
help(sys.stdin)

Help on TextIOWrapper object:

class TextIOWrapper(_TextIOBase)
 |  TextIOWrapper(buffer, encoding=None, errors=None, newline=None, line_buffering=False, write_through=False)
 |  
 |  Character and line based layer over a BufferedIOBase object, buffer.
 |  
 |  encoding gives the name of the encoding that the stream will be
 |  decoded or encoded with. It defaults to locale.getpreferredencoding(False).
 |  
 |  errors determines the strictness of encoding and decoding (see
 |  help(codecs.Codec) or the documentation for codecs.register) and
 |  defaults to "strict".
 |  
 |  newline controls how line endings are handled. It can be None, '',
 |  '\n', '\r', and '\r\n'.  It works as follows:
 |  
 |  * On input, if newline is None, universal newlines mode is
 |    enabled. Lines in the input can end in '\n', '\r', or '\r\n', and
 |    these are translated into '\n' before being returned to the
 |    caller. If it is '', universal newline mode is enabled, but line
 |    endings are return

In [113]:
def answer():
    # get result
    print('Some Result', end='\n')
# since, answer doesn't return a value, it's impossible to do regular unittest

In [114]:
answer()

Some Result


In [118]:
# using patch as a decorator
@patch('sys.stdout', new_callable=StringIO)
def test_answer(mock_stdout) -> None:
    answer()
    assert mock_stdout.getvalue() == 'Some Result\n'
    
    # print('all test passed') doesn't print to standard out

In [117]:
print('hi')

hi


In [119]:
test_answer()

In [122]:
# using context syntax
with patch('sys.stdout', new_callable=StringIO) as mock_stdout:
    answer()
    print('test')

    assert mock_stdout.getvalue() == 'Some Result\ntest\n'

In [123]:
print('hi')

hi


In [124]:
# standard input
import sys

def getData():
    x = sys.stdin.read()
    return x

def getLines():
    x = sys.stdin.readlines()
    return x

def getInput():
    x = input()
    return x

In [125]:
data = getInput()

In [126]:
data

'hello'

In [127]:
@patch('sys.stdin')
def test_getData(mock_stdin) -> None:
    mock_stdin.read.return_value = '2 3\n'
    data = getData()
    # use the data
    # get the result
    assert data == '2 3\n'
    # assert result

In [128]:
test_getData()

In [129]:
with patch('sys.stdin') as mock_stdin:
    mock_stdin.readlines.return_value = ['1 2\n', '3 4\n']
    data = getLines()
    print(data)
    assert data == ['1 2\n', '3 4\n']
    

['1 2\n', '3 4\n']


In [134]:
# patching the input() function
with patch('__main__.input') as mock_input:
    mock_input.return_value = '1 2 3 4'
    assert getInput() == '1 2 3 4'

### where to patch

- for patching to work, you must ensure that you patch the name used by the system under test
- must tell `patch()` correctly where to look for the object/name you want mocked
- if you choose the wrong target location, the result of `patch()` could be something you didn't expect
- good rule of thumb is to patch() the object where it is *looked up*

In [135]:
# datetime module is imported
import datetime
from unittest.mock import patch

In [136]:
# patch datetime class
with patch('datetime.datetime'):
    # datetime module is NOT patched
    print(datetime)
    # datetime class is patched MagicMock now
    print(datetime.datetime)

<module 'datetime' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/datetime.py'>
<MagicMock name='datetime' id='4567279264'>


In [137]:
# just the datetime class is imported
from datetime import datetime

In [138]:
# the patch has no effect because there's 
# unmocked datetime class imported into the current scope
with patch('datetime.datetime'):
    print(datetime)
    print(datetime.today())

<class 'datetime.datetime'>
2025-10-14 11:57:09.507544


In [139]:
# if you need to patch datetime class imported into the global namespace
# you do the following
with patch('__main__.datetime'):
    print(datetime)

<MagicMock name='datetime' id='4567159808'>


## common mocking problems

### changes to object interfaces and misspellings

- when interface of an object changes, any tests relying on a Mock of that object may become irrelevant
- misspelling can break a test; recall that Mock creates its interface when you access its members
    - you'll essentially create a new interface when you misspell a name

### changes to external dependencies

- when external dependency changes its interface, your Python objects will become invalid
- your tests will pass but the actual production code will fail

### avoiding common problems using specifications

- use `spec` parameter providing the list of valid interface/method names of module/class you're mocking

In [140]:
from unittest.mock import Mock
from unittest.mock import patch

In [141]:
# provide a list of valid api names of the class you want to mock 
calendar = Mock(spec=['is_weekday', 'get_holidays'])

In [142]:
calendar.is_weekday()

<Mock name='mock.is_weekday()' id='4567162064'>

In [143]:
# Mock raises AttributeError as create_event() is not in sepc
calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'

In [144]:
# automatically create specifications
from unittest.mock import create_autospec

from src.mocking import my_calendar

In [145]:
help(my_calendar)

Help on module src.mocking.my_calendar in src.mocking:

NAME
    src.mocking.my_calendar - A module for dealing with calendars.

FUNCTIONS
    get_holidays() -> Any
        Get the upcoming holidays.
        
        Returns:
            Any: The upcoming holidays
    
    is_weekday() -> bool
        Return True if today is a weekday, False otherwise.
        
        Returns:
            bool: True if today is a weekday, False otherwise

DATA
    Any = typing.Any
        Special type indicating an unconstrained type.
        
        - Any is compatible with every type.
        - Any assumed to have all methods.
        - All values assumed to be instances of Any.
        
        Note that all the above statements are true from the point of view of
        static type checkers. At runtime, Any should not be used with instance
        or class checks.

FILE
    /Users/rbasnet/Fall2025/Object-Oriented-Programming-Design-Patterns/notebooks/src/mocking/my_calendar.py




In [146]:
calendar = create_autospec(my_calendar)

In [147]:
calendar.is_weekday()

<MagicMock name='mock.is_weekday()' id='4567379536'>

In [149]:
calendar.sepc

AttributeError: Mock object has no attribute 'sepc'

In [148]:
calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'

In [151]:
# if you're using patch do the following
with patch('__main__.my_calendar', autospec=True) as calendar:
    calendar.is_weekday()
    calendar.get_holidays()
    #calendar.create_event()

## How much testing is enough?

- how much of your code is actutally being tested?
    - are the corner test cases generated by **hypothesis** enough to test every line, branch, result of your code?
- According to E. W. Dijkstra - "Program testing can be used to show the presence of bugs, but never to show their absence!"

### Code coverage

- with the useage of mocking/patching and hypothesis libraries, you should be able to get 100% code coverage with your unit tests without using #pragma: no cover

## Exercises

- solve the following Kattis problems using OOD
- must write adequate unittesting for the class API
- must write integration testing using Mock to simulate input and output for complete program testing
- use coverage to create html coverage report of your testing

1. FizzBuzz - https://open.kattis.com/problems/fizzbuzz
2. Mixed Fractions - https://open.kattis.com/problems/mixedfractions
